# This notebook aims to simulate available possibility of frac scenarios based on frac.py and emitted CO2 associated with these scenarios. Later, simulated big data can be used for predictive analytics, sensitivity analysis and scenario modeling

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm

In [3]:
#import of frac.py

import sys  
sys.path.insert(0, '/Users/anatolyaseev/Docs/CarbonTracker/')
import frac

# Current global variables for Monte Carlo



In [9]:
def monte_carlo_mgl(mc_sample_size = 50000):
    '''This is the function to generate monte carlo samples of global model paramters
        mc_sample_size: int, size of monte carlo samples, default = 100

        This Monte Carlo (MC) calculated for many constants and main inputs are:
        P = 13000 psi
        Q = 90 bar/min
        etta = 0.9
        t_tanks = 25
        t_pumps = 18

        For Prime Movers we used diesel and lng as a fuel, for transportation only diesel

    '''
    num_stage = np.random.randint(30,150, size = mc_sample_size)
    stage_time = np.random.randint(60,240, size = mc_sample_size)
    dis_fraction = np.random.uniform(0,1, size = mc_sample_size)
    dist_to_water = np.random.randint(0,200, size = mc_sample_size)
    truck_consumption = np.random.randint(10,25, size = mc_sample_size)
    water_per_stage = np.random.randint(100000,1000000, size = mc_sample_size)
    prop_per_stage = np.random.randint(10,100, size = mc_sample_size)
    dist_to_sand = np.random.randint(0,200, size = mc_sample_size)
    dist_to_water = np.random.randint(0,200, size = mc_sample_size)
    dist_to_fuel = np.random.randint(0,200, size = mc_sample_size)
    mob_distance = np.random.randint(1,500, size = mc_sample_size)
    sand_footprint = np.random.uniform(0,0.5, size = mc_sample_size)

    prior_mgl_samples = pd.DataFrame(data = 
                                     {'num_stage': num_stage, 
                                      'stage_time': stage_time, 
                                      'dis_fraction': dis_fraction,
                                      'dist_to_water' : dist_to_water,
                                      'truck_consumption': truck_consumption,
                                      'water_per_stage': water_per_stage,
                                      'prop_per_stage': prop_per_stage, 
                                      'dist_to_sand': dist_to_sand,
                                      'dist_to_water': dist_to_water,
                                      'dist_to_fuel': dist_to_fuel,
                                      'mob_distance': mob_distance,
                                      'sand_footprint': sand_footprint})
    
    prior_mgl_samples.to_csv("./"+str(mc_sample_size)+"_monte_carlo_parameters.csv", sep=',',index=False)
    print('Monte Carlo results of global parameters saved to file: "./' +str(mc_sample_size) + '_monte_carlo_parameters.csv"')

In [27]:
def monte_carlo_mdh(f_m_tablefile):
    # INPUTS
    fname_parameters = f_m_tablefile
    df = pd.read_csv(fname_parameters)
    fname = 'fracCO2' # Name your simulation
    # Define parameters
    num_stage = df['num_stage'].values
    stage_time = df['stage_time'].values
    dis_fraction = df['dis_fraction'].values
    truck_consumption = df['truck_consumption'].values
    water_per_stage = df['water_per_stage'].values
    prop_per_stage = df['prop_per_stage'].values
    dist_to_sand = df['dist_to_sand'].values
    dist_to_water = df['dist_to_water'].values
    dist_to_fuel = df['dist_to_fuel'].values
    mob_distance = df['mob_distance'].values
    sand_footprint = df['sand_footprint'].values

    
    nrealizations = len(df)

    CO2prediction = np.zeros(nrealizations)
    for i in tqdm(range(nrealizations)):
        F = frac.Frac('conv', stage_time[i], num_stage[i], 13000, 90, 25, 18)
        primeMover = F.primeMover(0, dis_fraction[i], 0, 1 - dis_fraction[i], 0)

        sandTransport = F.sandTransport(prop_per_stage[i], 24, dist_to_sand[i], truck_consumption[i])
        waterTransport = F.waterTransport(water_per_stage[i], 20, dist_to_water[i], truck_consumption[i])
        fuelTransport = F.fuelTransport(10, dist_to_fuel[i], truck_consumption[i])

        materialTransport = sandTransport + waterTransport + fuelTransport
        mobDemob = F.mobDemob(mob_distance[i], truck_consumption[i])
        land = F.land()
        sandMining = F.sandMining(prop_per_stage[i], sand_footprint[i])
        people  = F.people()
        auxillary = F.auxillary()

        CO2prediction[i] = primeMover + materialTransport + mobDemob + land + sandMining + people + auxillary
        
    return CO2prediction

In [28]:
sample_size = 50000
monte_carlo_mgl(mc_sample_size = sample_size)

Monte Carlo results of global parameters saved to file: "./50000_monte_carlo_parameters.csv"


In [35]:
f_m_tablefile = str(sample_size) + '_monte_carlo_parameters.csv'
fracCO2 = monte_carlo_mdh(f_m_tablefile)
np.save('fracCO2', fracCO2)

In [4]:
fracCO2 = np.load('fracCO2.npy')

In [7]:
import plotly.express as px

fracCO2_tons = fracCO2 / 1000
fig = px.histogram(fracCO2_tons, title = 'CO2 emissions from frac, tons')
fig.update_layout(showlegend=False)
fig.show()

# Sensitivity part. We take our table with all variables and check which one brings more uncertainty on overall frac CO2 emissions

In [8]:
from DGSA_Light.DGSA_light import DGSA_light
from DGSA_Light.gsa_pareto_plt import gsa_pareto_plt
import pandas as pd
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [46]:
# Load global model parameters
f_m_tablefile = pd.read_csv('50000_monte_carlo_parameters.csv')
f_m_tablefile.head()

,num_stage,stage_time,dis_fraction,dist_to_water,truck_consumption,water_per_stage,prop_per_stage,dist_to_sand,dist_to_fuel,mob_distance,sand_footprint
0,30,89,0.383013,51,19,639088,71,42,148,276,0.459036
1,38,119,0.755026,46,24,918962,69,147,196,304,0.026890
2,112,200,0.686339,33,10,754593,86,114,188,299,0.449080
3,108,208,0.071234,56,19,438149,80,79,101,365,0.464654
4,43,154,0.230675,45,20,191209,38,10,69,78,0.317510


In [48]:
global_vars = f_m_tablefile
names = global_vars.columns.values.tolist()

In [49]:
dgsa_global_variables_to_model = DGSA_light(global_vars.to_numpy(), model_var, ParametersNames = names)

['num_stage',
 'stage_time',
 'dis_fraction',
 'dist_to_water',
 'truck_consumption',
 'water_per_stage',
 'prop_per_stage',
 'dist_to_sand',
 'dist_to_fuel',
 'mob_distance',
 'sand_footprint']